In [1]:
pip install opencv-python mediapipe numpy scikit-learn joblib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import cv2
import numpy as np
import mediapipe as mp
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import joblib

In [3]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)
mp_draw = mp.solutions.drawing_utils

def collect_gesture_data(gesture_label, num_samples=300):
    cap = cv2.VideoCapture(0)
    print(f"Collecting data for: {gesture_label}")
    print("Press 'q' to quit...")
    
    data = []
    samples_collected = 0
    
    while samples_collected < num_samples:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(rgb_frame)

        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                landmarks = []
                for lm in hand_landmarks.landmark:
                    landmarks.extend([lm.x, lm.y, lm.z])
                
                data.append([landmarks, gesture_label])
                samples_collected += 1

        cv2.imshow("Collecting Data", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    print(f"Collected {samples_collected} samples for gesture: {gesture_label}")
    return data

In [4]:
gesture_labels = ["Fist", "Palm", "Thumbs Up", "Victory", "Pointing"]
all_data = []

for label in gesture_labels:
    gesture_data = collect_gesture_data(label)
    all_data.extend(gesture_data)

Press 'q' to quit...
Collected 300 samples for gesture: Fist
Press 'q' to quit...
Collected 300 samples for gesture: Palm
Press 'q' to quit...
Collected 300 samples for gesture: Thumbs Up
Press 'q' to quit...
Collected 300 samples for gesture: Victory
Press 'q' to quit...
Collected 300 samples for gesture: Pointing


In [5]:
import pickle

with open('gesture_data.pkl', 'wb') as file:
    pickle.dump(all_data, file)

print("Data saved successfully!")

Data saved successfully!


In [6]:
with open('gesture_data.pkl', 'rb') as file:
    data = pickle.load(file)

X = [item[0] for item in data]
y = [item[1] for item in data]

X = np.array(X)
y = np.array(y)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

classifier = RandomForestClassifier(random_state=42)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)
print(classification_report(y_test, y_pred))

# Save the model
joblib.dump(classifier, 'gesture_model.pkl')
print("Model saved successfully!")

              precision    recall  f1-score   support

        Fist       1.00      1.00      1.00        56
        Palm       1.00      1.00      1.00        65
    Pointing       1.00      1.00      1.00        64
   Thumbs Up       1.00      1.00      1.00        49
     Victory       1.00      1.00      1.00        66

    accuracy                           1.00       300
   macro avg       1.00      1.00      1.00       300
weighted avg       1.00      1.00      1.00       300

Model saved successfully!


In [8]:
gesture_model = joblib.load('gesture_model.pkl')

def recognize_gestures():
    cap = cv2.VideoCapture(0)
    print("Press 'q' to exit...")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame = cv2.flip(frame, 1)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(rgb_frame)

        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                landmarks = []
                for lm in hand_landmarks.landmark:
                    landmarks.extend([lm.x, lm.y, lm.z])
                
                landmarks = np.array(landmarks).reshape(1, -1)
                prediction = gesture_model.predict(landmarks)
                gesture_name = prediction[0]

                cv2.putText(frame, f"Gesture: {gesture_name}", (10, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        
        cv2.imshow("Gesture Recognition", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

recognize_gestures()

Press 'q' to exit...
